Note that the first run for a date range will include downloading and preprocessing global VIIRS data for that date range. 

Also note that the first run for a region in a date range will include preprocessing files for that region. 

In [ ]:
from fireatlas.FireRunDaskCoordinator import Run_local
from fireatlas.FireTypes import Region, TimeStep
import datetime as dt
import os
import subprocess
import cProfile
import traceback
import csv

import pandas as pd


def run_feds_with_logging(region: Region, tst: TimeStep, ted: TimeStep):
    
    from fireatlas import settings 

    run_start_time = dt.datetime.now()
    # start_timestamp = start_time.strftime("%Y%m%d_%H%M%S")

    os.makedirs("benchmark_logs", exist_ok=True)
    log_file = "./benchmark_logs/run_log.csv"

    run = 0
    
    if os.path.exists(log_file):
        with open(log_file, newline="") as f:
            reader = csv.DictReader(f)
            for row in reader:
                if region[0] in row["region"]:
                    run += 1

    run_name = f"{region[0]}_run{run}"
    profile_stats_filename = f"{run_name}.prof"


    log = {
        "run_name": run_name,
        "run_start_time": run_start_time.isoformat(),
        "run_end_time": "", 
        "duration_seconds": "",
        "region": region,
        "tst": tst,
        "ted": ted, 
        "git_branch": "", 
        "git_commit_hash": "", 
        "FIRE_SOURCE": settings.FIRE_SOURCE, 
        "N_DASK_WORKERS": settings.N_DASK_WORKERS,
        "FTYP_OPT": settings.FTYP_OPT, 
        "run_completed": False, 
        "error": "",
        "profile_file": profile_stats_filename
    }

    try:
        log["git_branch"] = subprocess.check_output(["git", "rev-parse", "--abbrev-ref", "HEAD"],stderr=subprocess.DEVNULL, text=True).strip()
        log["git_commit_hash"] = subprocess.check_output(["git", "rev-parse", "HEAD"],stderr=subprocess.DEVNULL, text=True).strip()
    except: 
        # if this doesnt work, whatever
        pass 

    profiler = cProfile.Profile()

    try:
        profiler.enable()
        _ = Run_local(region, tst, ted)
        profiler.disable()
        log["run_completed"] = True
    except Exception:
        profiler.disable()
        log["error"] = traceback.format_exc()
    
    run_end_time = dt.datetime.now()
    log["run_end_time"] = run_end_time.isoformat()
    log["duration_seconds"] = (run_end_time - run_start_time).total_seconds()

    os.makedirs("benchmark_profiles", exist_ok=True)
    profiler.dump_stats(f"./benchmark_profiles/{profile_stats_filename}")
    
    csv_file = "benchmark_logs/run_log.csv"
    exists = os.path.exists(csv_file)
    with open(csv_file, "a", newline="") as f:
        writer = csv.DictWriter(f, log.keys())
        if not exists:
            writer.writeheader()
        writer.writerow(log)
    
    print(f"[run_with_logging] Run logged to {csv_file}")
    print(f"[run_with_logging] Profile saved to {profile_stats_filename}")


In [ ]:
# careful! clears all local FEDS outputs
!rm -r ./data/FEDSoutput-v3/

In [ ]:
# test on one region 
# region = ["North_America", "North_America.geojson"]
# tst = [2024, 1, 1, "AM"]
# ted = [2024, 1, 2, "AM"]

# Run_local(region, tst, ted) # make sure it runs in general, without logging 
# run_feds_with_logging(region, tst, ted) # test on one region 

In [ ]:
# run one day for each region with logging 
# took 7 minutes for first run of jan 1-2 

regions = ["Africa",
           "Australia_New_Zealand",
           "Caribbean",
           "Central_Asia",
           "Europe_W_Siberia",
           "India", 
           "Japan_Far_East",
           "North_America",
           "SE_Asia", 
           "Seven_Seas", 
           "Siberia", 
           "South_America"]

for r in regions:
    region = [r, r+".geojson"]
    tst = [2024, 1, 1, "AM"]
    ted = [2024, 1, 2, "AM"]
    run_feds_with_logging(region, tst, ted)

In [ ]:
df=pd.read_csv("./benchmark_logs/run_log.csv")

df

In [ ]:
# add profiling to each run 

In [ ]:
# run one month for each region with logging 

In [ ]:
# set up monthly (?) checkpointing script for long runs 